In [1]:
import numpy as np
import cvxpy as cp
from scipy.io import loadmat

Load $Y_{bus}$ matrix from MATLAB

In [2]:
# Note: in per-unit
Y = loadmat('Ybus')['Ybus'].todense()

Problem parameters

In [3]:
# Number of buses
N = Y.shape[0]

# Susceptance matrix. Recall in DC power flow, G = 0
B = np.imag(Y)

# System base, in MVA
base = 100

# The load consumed at each bus
p_d = np.array([0, 40, 150, 80, 130])/base

# Matrix defining the cost of the generators
# The i,j element is the coefficient of the power produced at the ith generator raised to the j-1 power
# Copied from PowerWorld
C = np.array(
    [
        [373.5, 10, 0.016],
        [403.6, 8, 0.018],
        [253.2, 12, 0.018]
    ]
)

# Minimum and maximum generator outputs, taken from PowerWorld
p_min = np.array([100, 150, 0])/base
p_max = np.array([400, 500, 300])/base

# The buses with generators and loads
G = [1, 2, 4]
L = [2, 3, 4, 5]

First, we'll solve the normal supply-side OPF. Re-index the problem such that the generator buses preceed the non-generator buses. Within each block, keep the order the same.

In [4]:
NG = [n for n in range(1,N+1) if n not in G]
idx = np.array(G+NG)-1
B = B[idx][:,idx]
p_d = p_d[idx]

Solve DC OPF. Line flow limits not yet implmented, but they are not binding for this example. Confirm that the total cost is consistent with PowerWorld.

In [5]:
p_g = cp.Variable(len(G))
delta = cp.Variable(N)
cp.Problem(
    cp.Minimize(cp.vec(C.T)@cp.vec(cp.vstack([(base*p_g)**n for n in range(3)]))),
    [
        p_g >= p_min,
        p_g <= p_max,
        cp.hstack([p_g,np.zeros(len(NG))])-p_d == -B@delta,
        cp.sum(p_g) == p_d.sum()
    ]
).solve()

5840.78888888889

Now let's solve a demand-side problem with zero marginal cost, fixed supply, and elastic demand. For this minimal example, we make the follwing assumptions:
- Each load submits a demand curve. The system operator accepts the maximum-price bid until the renewable output is met.
- Each load can consume as much power as desired.
- For each load, elasticity is constant. That is, the marginal benefit curve for load $i$ is given by $MB_i(P)=K_iP^{e_i}$, where $-1<e_i<0$ is the elasticity. Note that this formulation insures that marginal benefit is always positive.

For simplicity, suppose each generator supplies its optimal output from the supply-side problem.

In [6]:
p_g = np.hstack([p_g.value, np.zeros(len(NG))])

Again, reindex the system, this time with the load buses preceding the non-load buses

In [7]:
B = B[idx][:,idx]
p_d = p_d[idx]
NL = [n for n in range(1,N+1) if n not in L]
idx = np.array(L+NL)-1
B = B[idx][:,idx]
p_g = p_g[idx]
p_d = p_d[idx]

For simplicity, we choose $e_1=\dots=e_L=-0.2$. Furthermore, choose $K_i$ by taking the marginal benefit vector to be $MB(P_d^*)=\lambda\mathbf{1}$ where $P_d^*$ is the load vector given by PowerWorld and $\lambda$ is the system marginal price at optimality (copied from PowerWorld).

In [8]:
e = -0.2
price = 13.68
K = base*p_d[:len(L)]**e/price

Solve!

In [9]:
p_d = cp.Variable(len(L))
delta = cp.Variable(N)
cp.Problem(
    cp.Maximize(cp.sum(cp.multiply(K,(base*p_d)**(e+1)))),
    [
        p_d >= 0,
        p_g-cp.hstack([p_d,np.zeros(len(NL))]) == -B@delta,
        cp.sum(p_d) == p_g.sum()
    ]
).solve()

1226.1022239383628

The total benefit (above) and the consumption vector (below) are consistent in magnitude with the supply-side problem.

In [10]:
p_d.value*100

array([192.81677209,  51.42545795,  96.42086215,  59.33690782])